
Automatic question generation using the Korean T5 model, automatically generate question sentences with given words as answers.

Install libraries

In [4]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

!pip install tqdm

!pip install wandb

!sudo apt-get install git-lfs

Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
[sudo] password for josh: 

In [3]:
import logging
import os
#import KoBart
#import dataset
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from tqdm import tqdm
import math

2023-05-04 19:52:44.675391: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-04 19:52:44.694173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 19:52:45.019672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [6]:
# Config
batch_size = 16
epochs = 5
warmup_ratio = 0.1
learning_rate = 1e-04
grad_clip = 1.0
train_log_interval = 100
validation_interval = 2000
save_interval = 2000

In [1]:
# Model, tokenizer init
# model = KoBart.KoBARTConditionalGeneration()
# tokenizer = model.tokenizer

NameError: name 'kobart' is not defined

In [8]:
# Data file path
train_path = './data/train.tsv'
dev_path = './data/dev.tsv'

In [9]:
from tokenizers import Encoding
# dataset, dataloader
train_dataset = dataset.KoBARTQGDataset(train_path, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size)

dev_dataset = dataset.KoBARTQGDataset(dev_path, tokenizer)
dev_dataloader = DataLoader(dev_dataset, batch_size)

In [10]:
# optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, correct_bias=False)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
# scheduler
data_len = len(train_dataloader)
num_train_steps = int(data_len / batch_size * epochs)
num_warmup_steps = int(num_train_steps * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

In [12]:
# logging data info
logging.info(f'data length {data_len}')
logging.info(f'num_train_steps : {num_train_steps}')
logging.info(f'num_warmup_steps : {num_warmup_steps}')

INFO:root:data length 3776
INFO:root:num_train_steps : 1180
INFO:root:num_warmup_steps : 118


In [13]:
# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
# dev
def _validate(
        model: KoBart.KoBARTConditionalGeneration,
        dev_dataloader: DataLoader,
        device: torch.device,
        logger: logging.Logger,
        global_step: int,
):
    model.eval()
    loss_list = []
    for batch_data in tqdm(dev_dataloader, desc="[EVAL]"):
        with torch.no_grad():
            input_ids, decoder_input_ids, labels = tuple(value.to(device) for value in batch_data.values())
            model_outputs = model.forward(input_ids, decoder_input_ids, labels)
            loss_list.append(model_outputs.loss.item())

    mean_loss = np.mean(loss_list)
    logger.info(f"[EVAL] global_step:{global_step} loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}")
    model.train()

    return mean_loss

In [15]:
model.train()

KoBARTConditionalGeneration(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(30000, 768, padding_idx=3)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(30000, 768, padding_idx=3)
        (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_fea

In [16]:
loss_list_between_log_interval = []
for epoch_id in range(epochs):
    for step_index, batch_data in tqdm(enumerate(train_dataloader), f"[TRAIN] EP:{epoch_id}", total=len(train_dataloader)):
            global_step = len(train_dataloader) * epoch_id + step_index + 1
            optimizer.zero_grad()

            input_ids, decoder_input_ids, labels = tuple(value.to(device) for value in batch_data.values())

            model_outputs = model.forward(input_ids, decoder_input_ids, labels)

            model_outputs.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
            scheduler.step()

            # for logging
            loss_list_between_log_interval.append(model_outputs.loss.item())

            if global_step % train_log_interval == 0:
                mean_loss = np.mean(loss_list_between_log_interval)
                logger.info(
                    f"EP:{epoch_id} global_step:{global_step} "
                    f"loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}"
                )
                loss_list_between_log_interval.clear()

            if global_step % validation_interval == 0:
                dev_loss = _validate(model, dev_dataloader, device, logger, global_step)

            if global_step % save_interval == 0:
                state_dict = model.state_dict()
                model_path = os.path.join('./output', f"kobart_step_{global_step}.pth")
                logger.info(f"global_step: {global_step} model saved at {model_path}")
                torch.save(state_dict, model_path)

[TRAIN] EP:0:  53%|█████▎    | 1999/3776 [18:24<16:23,  1.81it/s]INFO:root:EP:0 global_step:2000 loss:2.1050 perplexity:8.2071

[EVAL]: 100%|██████████| 361/361 [01:11<00:00,  5.03it/s]
INFO:root:[EVAL] global_step:2000 loss:2.0929 perplexity:8.1085
INFO:root:global_step: 2000 model saved at ./output/kobart_step_2000.pth
[TRAIN] EP:1:   6%|▌         | 223/3776 [02:03<32:57,  1.80it/s]INFO:root:EP:1 global_step:4000 loss:1.4689 perplexity:4.3447

[EVAL]: 100%|██████████| 361/361 [01:11<00:00,  5.02it/s]
INFO:root:[EVAL] global_step:4000 loss:2.0573 perplexity:7.8247
INFO:root:global_step: 4000 model saved at ./output/kobart_step_4000.pth
[TRAIN] EP:1:  59%|█████▉    | 2223/3776 [21:43<14:18,  1.81it/s]INFO:root:EP:1 global_step:6000 loss:1.2977 perplexity:3.6607

[EVAL]: 100%|██████████| 361/361 [01:11<00:00,  5.02it/s]
INFO:root:[EVAL] global_step:6000 loss:1.8516 perplexity:6.3702
INFO:root:global_step: 6000 model saved at ./output/kobart_step_6000.pth
[TRAIN] EP:2:  12%|█▏        | 4